<a href="https://colab.research.google.com/github/athishr88/NN_DL/blob/main/LA4/LA4_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VQA 

Data preprocessing

In [ ]:
## Set up the VizWiz-VQA dataset

img_dir = "https://vizwiz.cs.colorado.edu//VizWiz_visualization_img/"
splits = ["train", "val", "test"]
dataset_files_dict = {}

for split in splits:
  annotation_file = "https://ivc.ischool.utexas.edu/VizWiz_final/vqa_data/Annotations/%s.json" %split
  dataset_files_dict[split] = annotation_file

dataset_files_dict

{'test': 'https://ivc.ischool.utexas.edu/VizWiz_final/vqa_data/Annotations/test.json',
 'train': 'https://ivc.ischool.utexas.edu/VizWiz_final/vqa_data/Annotations/train.json',
 'val': 'https://ivc.ischool.utexas.edu/VizWiz_final/vqa_data/Annotations/val.json'}

In [ ]:
# Read the files to extract train, test, val dataset with label (Only 20% of data will be used in this exercise)
import requests
import numpy as np
import pandas as pd
import random

fraction_of_data = 0.04

def extract_dataset(split):
  split_data = requests.get(dataset_files_dict[split], allow_redirects=True)
  data_all = split_data.json()
  random.shuffle(data_all)
  data = data_all[:int(len(data_all)*fraction_of_data)]
  data = pd.DataFrame(data)
  return data

train_data_raw = extract_dataset("train")
val_data_raw = extract_dataset("val")
test_data_raw = extract_dataset("test")

In [ ]:
# Visualize train_data_raw
train_data_raw.head()

,image,question,answers,answer_type,answerable
0,VizWiz_train_00018154.jpg,That you very much. I really appreciate it.,"[{'answer': 'unanswerable', 'answer_confidence...",unanswerable,0
1,VizWiz_train_00001855.jpg,>Label on this bottle.,"[{'answer_confidence': 'yes', 'answer': 'machi...",unanswerable,0
2,VizWiz_train_00000950.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'unans...",other,1
3,VizWiz_train_00009934.jpg,What is this?,"[{'answer_confidence': 'no', 'answer': 'unsuit...",other,1
4,VizWiz_train_00007828.jpg,Can you tell me what this bottle says?,"[{'answer_confidence': 'yes', 'answer': 'unsui...",unanswerable,0


In [ ]:
# Remove small amount of unanswerable data (Class balancing)

def balance_data(data):
  data_answerable = data[data.answerable != 0]
  data_unanswerable = data[data.answerable == 0]
  data_unanswerable_sample = data_unanswerable.sample(frac=.1)

  balanced_data = pd.concat([data_answerable, data_unanswerable_sample], ignore_index = True)
  balanced_data.reset_index()
  return balanced_data

train_data_balanced = balance_data(train_data_raw)
val_data_balanced = balance_data(val_data_raw)


In [ ]:
## Create labels
# Only the most frequent answer out of the 10 answers is considered as label

def most_frequent(List):
    return max(set(List), key = List.count)

def generate_labels(df):
  labels = []
  gtlist = df["answers"]

  for answers in gtlist:
    all_answers = []
    for answer in answers:
      all_answers.append(answer["answer"])
    labels.append(most_frequent(all_answers))
  
  labels_text = [x.split()[0] for x in labels]
  df["label"] = labels_text
  
  return df

train_data_w_labels = generate_labels(train_data_balanced)
val_data_w_labels = generate_labels(val_data_balanced)

In [ ]:
train_data_w_labels.head()

,image,question,answers,answer_type,answerable,label
0,VizWiz_train_00000950.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'unans...",other,1,unanswerable
1,VizWiz_train_00009934.jpg,What is this?,"[{'answer_confidence': 'no', 'answer': 'unsuit...",other,1,tissue
2,VizWiz_train_00009446.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'gerg'...",other,1,raisins
3,VizWiz_train_00018054.jpg,What is the color of the sky in this picture?,"[{'answer_confidence': 'yes', 'answer': 'blue'...",other,1,blue
4,VizWiz_train_00002123.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'pepsi...",other,1,pepsi


In [ ]:
# Modulizing

train_images = train_data_w_labels["image"] 
train_questions = train_data_w_labels["question"]
train_labels_text = train_data_w_labels["label"]

val_images = val_data_w_labels["image"] 
val_questions = val_data_w_labels["question"]
val_labels_text = val_data_w_labels["label"]

test_images = test_data_raw["image"]
test_questions = test_data_raw["question"]

print(len(train_images), len(train_labels_text))

621 621


In [ ]:
None in test_questions

False

In [ ]:
# Check how many unique classes are there in training set
print("Classes: ", len(np.unique(train_labels_text)))

Classes:  340


In [ ]:
# Taking the most common 500 labels
from collections import Counter

Counter = Counter(train_labels_text)
most_occur = Counter.most_common(500)
classes = [x[0] for x in most_occur]
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((i, c) for i, c in enumerate(classes))
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])

## Convert the train labels to corresponding int values
train_labels = names_to_ids(train_labels_text)
val_labels = names_to_ids(val_labels_text)

In [ ]:
# Additional class "unknown" is defined to accomodate unknown classes

class_to_index["unknown"] = len(class_to_index)-1
index_to_class[len(class_to_index)-1] = "unknown"

val_labels[val_labels==None] = len(class_to_index)-1
train_labels[train_labels==None] = len(class_to_index)-1

In [ ]:
## Create our Text Vectorizer to index our vocabulary based on the train samples 
from keras.layers import TextVectorization
import tensorflow as tf

vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=100)
text_ds = tf.data.Dataset.from_tensor_slices(train_questions).batch(128) ## Read batches of 128 samples
vectorizer.adapt(text_ds)

In [ ]:
## Create a map to get the unique list of the vocabulary
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
# Prepare vectorized train, val and test data

x_train_questions = vectorizer(np.array([[s] for s in train_questions])).numpy()
x_val_questions = vectorizer(np.array([[s] for s in val_questions])).numpy()
x_test_questions = vectorizer(np.array([[s] for s in test_questions])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [ ]:
# Preparing image data
import cv2
from skimage import io

def generate_image_dataset(urls):
  print(len(urls))
  image_data = []
  for i, image_name in enumerate(urls):
    image_url = img_dir + image_name
    image = io.imread(image_url)
    image = cv2.resize(image, (224, 224))
    image = image/255.0
    image_data.append(image)
    print('\r', i, end='')
  return image_data

x_train_images = generate_image_dataset(train_images)
x_val_images = generate_image_dataset(val_images)
x_test_images = generate_image_dataset(test_images)
print(len(x_train_images), len(x_val_images))

621
 620131
 130320
 319621 131


In [ ]:
# Convert to tensor
import tensorflow as tf

x_train_imgs = tf.convert_to_tensor(np.array(x_train_images), dtype=tf.float32)
x_val_imgs = tf.convert_to_tensor(np.array(x_val_images), dtype=tf.float32)
x_test_imgs = tf.convert_to_tensor(np.array(x_test_images), dtype=tf.float32)
y_train = tf.convert_to_tensor(np.array(y_train), dtype=tf.float32)
y_val = tf.convert_to_tensor(np.array(y_val), dtype=tf.float32)

In [ ]:
## Download and unzip the Stanford GloVe model (pretrained word embeddings)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-03-31 20:34:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-31 20:34:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-31 20:34:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
## Read the embeddings in the pretrained model (we are using the 100D version of GloVe)
import os
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
## Create "embedding_matrix" to index our vocabulary using the GloVe model 
num_tokens = len(voc) 
embedding_dim = 100 ## 100 dimensions
hits = 0 ## number of words that were found in the pretrained model
misses = 0 ## number of words that were missing in the pretrained model

# Prepare embedding matrix for our word list
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 537 words (10 misses)


In [ ]:
## Define our embedding layer for the training model 
## We load our embedding_matrix as the initializer and set trainable to False to avoid retraining this layer

from keras.layers import Embedding, Concatenate
from keras.initializers import Constant

embedding_layer = Embedding(num_tokens, embedding_dim,
                            embeddings_initializer= Constant(embedding_matrix), 
                            trainable=False,
)

Model

In [ ]:
# Generate image features from pretrained VGG16 trained on Imagenet
from keras.applications.vgg16 import VGG16
from keras.regularizers import l2
from keras.layers import BatchNormalization

l2_value = l2(0.00001)

def image_model():
  image_input = Input(shape=(224,224,3), dtype="float32")
  vgg = VGG16()
  x = vgg(image_input)
  x = BatchNormalization()(x)
  return image_input, x

def question_model(dropout_rate=0.2):
  int_sequences_input = Input(shape=(None,), dtype="float32")
  embedded_sequences = embedding_layer(int_sequences_input)
  x = layers.Bidirectional(layers.LSTM(20, return_sequences=True))(embedded_sequences)
  x = Dropout(dropout_rate)(x)
  x = layers.Bidirectional(layers.LSTM(20))(x)
  x = BatchNormalization()(x)
  x = layers.Dense(1000, activation="relu", kernel_regularizer=l2_value)(x)
  preds = BatchNormalization()(x)
  # model = Model(int_sequences_input, preds)
  return int_sequences_input, preds

In [ ]:
# Generate image features from pretrained VGG16 trained on Imagenet

from keras import layers, Input, Model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Flatten, Embedding, Multiply, Conv2D, MaxPool2D, Embedding
import keras,os
import numpy as np
from keras.initializers import Constant

def VQA_model(dropout_rate=0.2, num_classes=500, num_layers=4):
    img_input, img_output = image_model()
    lstm_input, lstm_output = question_model()
    print("Merging final model...")
    x = tf.keras.layers.Concatenate(axis=1)([img_output, lstm_output])
    # print(x)
    x = Dense(1000, activation='relu', kernel_regularizer=l2_value)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(1000, activation='relu', kernel_regularizer=l2_value)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(1000, activation='relu', kernel_regularizer=l2_value)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    if num_layers > 4:
      x = Dense(1000, activation='relu', kernel_regularizer=l2_value)(x)
      x = BatchNormalization()(x)
      x = Dropout(dropout_rate)(x)

    preds = Dense(num_classes, activation='softmax')(x)
    model = Model((img_input, lstm_input), preds)
    # print(model.summary())
    return model

In [ ]:
from tensorflow import keras

dropout_values = [0.2, 0.3]
num_layers = [4, 5]

for do_value in dropout_values:
  for num_layer in num_layers:

    print(f"Model: Dropout value {do_value}, Number of Layers {num_layer}")

    vqa_model = VQA_model(dropout_rate=do_value, num_layers=num_layer)
    # opt = keras.optimizers.Adam(learning_rate=0.0001)

    vqa_model.compile(optimizer="adam", loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
    
    vqa_model.fit((x_train_imgs, x_train_questions), y_train, epochs=6,
                  batch_size=20, validation_data=((x_val_imgs, x_val_questions),
                                                  y_val))

Model: Dropout value 0.2, Number of Layers 4
Merging final model...
Epoch 1/6
32/32 [==============================] - 19s 388ms/step - loss: 6.6623 - accuracy: 0.0515 - val_loss: 6.1865 - val_accuracy: 0.0534
Epoch 2/6
32/32 [==============================] - 11s 342ms/step - loss: 5.7878 - accuracy: 0.0821 - val_loss: 6.2228 - val_accuracy: 0.0534
Epoch 3/6
32/32 [==============================] - 11s 344ms/step - loss: 5.1314 - accuracy: 0.1127 - val_loss: 6.1577 - val_accuracy: 0.0763
Epoch 4/6
32/32 [==============================] - 11s 345ms/step - loss: 4.7555 - accuracy: 0.0821 - val_loss: 6.2808 - val_accuracy: 0.0763
Epoch 5/6
32/32 [==============================] - 11s 347ms/step - loss: 4.4020 - accuracy: 0.1304 - val_loss: 6.4704 - val_accuracy: 0.0763
Epoch 6/6
32/32 [==============================] - 11s 348ms/step - loss: 4.1684 - accuracy: 0.1546 - val_loss: 6.5871 - val_accuracy: 0.0763
Model: Dropout value 0.2, Number of Layers 5
Merging final model...
Epoch 1/6
32